In [113]:
import pandas as pd
import seaborn as sns

In [114]:
def read_file_csv(file_name):
    return pd.read_csv(file_name, low_memory=False, encoding="ISO-8859-1")

In [115]:
books_data = "./books.csv"
ratings_data = "./ratings.csv"
pandas_data_books = read_file_csv(books_data)
pandas_data_ratings = read_file_csv(ratings_data)

In [116]:
ratings_data = pd.merge(pandas_data_books[['book_id', 'title']], pandas_data_ratings, on='book_id')
ratings_data.head()

,book_id,title,user_id,rating
0,1,"The Hunger Games (The Hunger Games, #1)",3634,4
1,1,"The Hunger Games (The Hunger Games, #1)",439,3
2,1,"The Hunger Games (The Hunger Games, #1)",3901,4
3,1,"The Hunger Games (The Hunger Games, #1)",3978,4
4,1,"The Hunger Games (The Hunger Games, #1)",3836,4


In [117]:
title_df = ratings_data[['book_id', 'title']].drop_duplicates(subset='book_id')
title_df.head()

,book_id,title
0,1,"The Hunger Games (The Hunger Games, #1)"
1764,2,Harry Potter and the Sorcerer's Stone (Harry P...
3398,3,"Twilight (Twilight, #1)"
4625,4,To Kill a Mockingbird
6372,5,The Great Gatsby


In [118]:
nb_voters_book = ratings_data['book_id'].value_counts()
nb_voters_df = pd.DataFrame(data={'book_id': nb_voters_book.index.tolist(), 'counts': nb_voters_book.values.tolist()})
nb_voters_df.head()

,book_id,counts
0,1,1764
1,4,1747
2,5,1655
3,2,1634
4,26,1597


In [119]:
rating_mean = ratings_data.groupby(['book_id'])['rating'].mean()
rating_mean_df = pd.DataFrame(data={'book_id': rating_mean.index.tolist(), 'mean_rating': rating_mean.values.tolist()})
rating_mean_df.head()

,book_id,mean_rating
0,1,4.238662
1,2,4.058140
2,3,3.376528
3,4,4.364625
4,5,3.748036


In [120]:
# Fix the number of minimum readers to get into the list - m
m = nb_voters_book.quantile(0.90)
print(m)

202.0


In [121]:
# Rating mean (all of the books) - C.
rating_all_mean = ratings_data['rating'].mean()
print(rating_all_mean)

3.779077570255898


In [122]:
title_voters_df = pd.merge(title_df, nb_voters_df, on=['book_id'], how='inner')
df = pd.merge(title_voters_df, rating_mean_df, on=['book_id'], how='inner')
df.head()

,book_id,title,counts,mean_rating
0,1,"The Hunger Games (The Hunger Games, #1)",1764,4.238662
1,2,Harry Potter and the Sorcerer's Stone (Harry P...,1634,4.058140
2,3,"Twilight (Twilight, #1)",1227,3.376528
3,4,To Kill a Mockingbird,1747,4.364625
4,5,The Great Gatsby,1655,3.748036


In [123]:
df['weighted_average_rating'] = ((df['counts'] / (df['counts'] + m)) * df['mean_rating']) + ((m / (df['counts'] + m)) * rating_all_mean)
df.head()

,book_id,title,counts,mean_rating,weighted_average_rating
0,1,"The Hunger Games (The Hunger Games, #1)",1764,4.238662,4.191441
1,2,Harry Potter and the Sorcerer's Stone (Harry P...,1634,4.058140,4.027437
2,3,"Twilight (Twilight, #1)",1227,3.376528,3.433432
3,4,To Kill a Mockingbird,1747,4.364625,4.303937
4,5,The Great Gatsby,1655,3.748036,3.751413
